In [144]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
from platform import python_version

In [145]:
Nimply = namedtuple("Nimply", "row, num_objects") #move

In [146]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
    

### Task 3.1 - Fixed rules based on nim-sum (an expert system)


In [149]:
def nim_sum(nim): #hard coded optimal agent

  X = nim._rows[0]

  # X is the nim_sum(bitwise xor) of all heap sizes
  for i in range(1, len(nim._rows)):
    X = X ^ nim._rows[i]

  nim_sum_val = []

  # calculate the nim_sum between X and each heap size
  for i in nim._rows:
    val = i ^ X
    nim_sum_val.append(val)

  row = "false"
 
  for i in range(len(nim_sum_val)):
    if nim_sum_val[i] < nim._rows[i]:
      row = i
      break
  
  # reduce that heap to value nim_sum
  if (row != "false"):
    num_objects = nim._rows[row] - nim_sum_val[row]
    move = Nimply(row, num_objects)
  
  else:
    rand_row = random.randrange(0,len(nim._rows))

    while(nim._rows[rand_row] == 0):
      rand_row = random.randrange(0,len(nim._rows))

    if(nim._rows[rand_row] != 1):
      rand_obj = random.randrange(1, nim._rows[rand_row])
                                  
    else: rand_obj = 1

    move = Nimply(rand_row, rand_obj)
  
  return move

In [150]:
logging.getLogger().setLevel(logging.DEBUG)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = nim_sum(nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 6 19 21>
DEBUG:root:status: After player 1 -> <1 3 5 7 2 11 13 15 6 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 2 0 13 15 6 19 21>
DEBUG:root:status: After player 1 -> <1 3 5 7 2 0 13 6 6 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 2 0 4 6 6 19 21>
DEBUG:root:status: After player 1 -> <1 1 5 7 2 0 4 6 6 19 21>
DEBUG:root:status: After player 0 -> <1 1 5 5 2 0 4 6 6 19 21>
DEBUG:root:status: After player 1 -> <1 1 5 5 1 0 4 6 6 19 21>
DEBUG:root:status: After player 0 -> <1 1 5 5 1 0 4 5 6 19 21>
DEBUG:root:status: After player 1 -> <1 1 5 5 1 0 4 5 6 19 11>
DEBUG:root:status: After player 0 -> <1 1 5 5 1 0 4 5 6 13 11>
DEBUG:root:status: After player 1 -> <1 1 5 5 1 0 3 5 6 13 11>
DEBUG:root:status: After player 0 -> <1 1 2 5 1 0 3 5 6 13 11>
DEBUG:root:status: After player 1 -> <1 1 2 1 1 0 3 5 6 13 11>
DEBUG:root:status: After player 0 -> <1 1 

### Task 3.2 - Evolved Rules

In [139]:
# variables
OFFSPRING_SIZE = 200
K= 5
POPULATION_SIZE = 50
TOURNAMENT_SIZE = 5

In [100]:
def pure_random(state: Nim) -> Nimply: #function from the professor
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [101]:
def nim_sum(state: Nim) -> int: #function from the professor
    *_, result = accumulate(state.rows, xor)
    return result

In [102]:
def active_rows_index(state):
    
    active_rows_index = []
    idx = 0
    
    for o in state.rows:
        if o > 0:
          active_rows_index.append(idx)
        idx += 1
        
    return active_rows_index

In [108]:
def cook_status(state): #part of the function is from the professor
    
    cooked = dict()
    
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ] #possible moves
    
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows) #number of rows that are not 0
    cooked["active_rows_index"] = active_rows_index(state) #get the index of rows that are not 0
    
    #get the index and the value of rows that only have one elem
    cooked["rows_with_one_element"] = [(index, r) for index, r in enumerate(state.rows) if r == 1] 
    
    #get the index and the value of rows that have more than one elem
    cooked["rows_multiple_elem"] = [(index, r) for index, r in enumerate(state.rows) if r > 1]
    
    #get the shortest row
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    
    #get the long row
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    
    
    cooked["nim_sum"] = nim_sum(state) #get the best move
    
    cooked["pure_random"] = pure_random(state) #get a random move

    brute_force = list()
    
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
        
    cooked["brute_force"] = brute_force

    return cooked


In [104]:
def optimal_startegy(state: Nim) -> Nimply: #function from the professor
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [105]:
def random_agent(state): #An agent returning a random move
    data = cook_status(state)
    return data["pure_random"]

In [106]:
def dumb(state): #A dumb agent that always picks one element from the longest row
    data = cook_status(state)
    row = data["longest_row"]
    return Nimply(row, 1)


In [124]:
OPPONENT = [optimal_startegy, random_agent, dumb] #opponents 

**Rules**

For a detailed explanation, please consult the README!

In [109]:
#Rule 1

def one_row_left(state, data, genome): #Rule 1 - if only one row left: leave x sticks
    
    
    active_row = data["active_rows_index"][0]
    elem_last_row = state.rows[active_row] #active_rows_index returns a list -> need to get the first one
   
    if elem_last_row < genome['Rule1']: #if the rule want to leave more sticks than exists in row
        ply = Nimply(active_row, 1)
        
    else:
        
        elem_to_remove = max(elem_last_row - genome["Rule1"], 1) #if take more elem than exists -> take all
        ply = Nimply(active_row, elem_to_remove)

    return ply

In [110]:
def rule_one_multiple_left(state, data, genome, rule): #Rule 2 and 4 
    
  active_rows_index = data["active_rows_index"]
  single_elem_rows = data["rows_with_one_element"]
  multiple_elem_rows = data["rows_multiple_elem"]


  if genome[rule][0] == 0:  # take from row with one elem
    
    elem = 1 # want to take the last elem in row
    single_elem_row = single_elem_rows[0][0] # looks like [(row,elem),(row,elem)] if two rows with single elem
    ply = Nimply(single_elem_row, 1) #takes from the first row that has 1 elem

  else: #take from the row with more than one element

    if len(multiple_elem_rows) == 0: #if single elem in all rows
      row = single_elem_rows[0][0]
      elem = 1
        
    else: 
      row = multiple_elem_rows[0][0] # [(row,elem)] -> since only one row with multiple elem
    
    if (genome[rule][1] > state.rows[row]): # if it wants to leave more elem than exists in row
      elem = 1
                        
    else: 
        
      elem = max(state.rows[row] - genome[rule][1], 1)
    
    ply = Nimply(row, elem)
      
  return ply


In [111]:
def rule_several_multiple_left(state, data, genome, rule): #Rule 3 and 5

  if (genome[rule][0] == 0): # choose from row with fewest elemt
    row = data['shortest_row']
    
  else: #choose from row with biggest elem
    row = data['longest_row']
      
  elem = max(state.rows[row] - genome[rule][1], 1)
  ply = Nimply(row, elem)

  return ply

In [112]:
def even_rows_left(state, data, genome): #rules for when we have an even number of non-zero rows
    
  rows_multiple_elem = data["rows_multiple_elem"]

  if len(rows_multiple_elem) == 1: # Rule 2 - only one row with multiple elems
    
    ply = rule_one_multiple_left(state, data, genome,'Rule2')
  
  else: # Rule 3 - more than one row with multiple elems 
           
    ply = rule_several_multiple_left(state, data, genome,'Rule3')

  return ply


In [115]:
def odd_number_of_rows_left(state, data, genome):  #rules for when we have an odd number of non-zero rows
    
    rows_multiple_elem = data["rows_multiple_elem"]

    if len(rows_multiple_elem) == 1: # Rule 4 - only one row with multiple elems
        ply = rule_one_multiple_left(state, data, genome,'Rule4')
  
    else: # Rule 5 - more than one row with multiple elems             
        ply = rule_several_multiple_left(state, data, genome,'Rule5')

    return ply

In [116]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        
        data = cook_status(state)

        active_rows_number = data["active_rows_number"]

        #only one active row left - we choose Rule 1
        if active_rows_number == 1:
            ply = one_row_left(state=state,data=data, genome=genome) 
        
        #even number of rows left - we choose Rule 2 or 3
        elif active_rows_number %2 == 0: # even numbers

            ply = even_rows_left(state=state, data=data, genome=genome)

        #odd number of rows left - we choose Rule 4 or 5
        else: 
            ply = odd_number_of_rows_left(state=state, data=data, genome=genome)

        return ply

    return evolvable

**Genetic functions**

For a detailed explanation of the genetic strategy, please consult the README!

In [118]:
def head2head(individual1, opponent, nim_size): #compute the fitness - makes two players compete against each other
    won = 0
    
    nim = Nim(nim_size) #creates the board

    players = (make_strategy(individual1), opponent) #gets the players
    player = 0
    
    while nim:
        
        ply = players[player](nim)
        nim.nimming(ply)
        player = 1 - player
        
    winner = 1 - player
    
    if winner == 0:
        return 1
    else:
        return 0


In [126]:
def calculate_fitness(population, nim_size): # calculate fitness for whole population
    NUM_MATCHES = 10
    
    for p1 in population:
        
        fitness = []

        for strat in OPPONENT: #plays against optimal player, random player and dumb player 
            wins = 0
            
            for _ in range(NUM_MATCHES):
                wins += head2head(p1, strat, nim_size)
                
            fitness.append(wins/NUM_MATCHES)
            
        p1['fitness'] = (fitness[0], fitness[1], fitness[2]) #fitness according to each type of player

In [137]:
def init_population(nim_size): #initiates population
    
    population = []
    max_leave = (nim_size-1)*2 # last row of the table will have nim_size*2-1 objects

    cond = POPULATION_SIZE
    
    while cond: #creates individuals
        individual = {'Rule1': random.randint(0,max_leave), 'Rule2': [random.randint(0,1), max_leave], 'Rule3': [random.randint(0, 1), random.randint(0, max_leave)],
        'Rule4': [random.randint(0,1), max_leave], 'Rule5': [random.randint(0,1), max_leave]}
        
        individual['fitness'] = ()
        population.append(individual)
        
        cond -= 1
        
    return population

In [127]:
def k_fittest_indv(population, k): #gets the k fittest individuals of a population
    return sorted(population, key=lambda indv: indv['fitness'], reverse=True)[:k]

In [129]:
# select k random individuals, return the one with best fitness
def tournament(population, nim_size):    
    
    contestors = random.sample(population, TOURNAMENT_SIZE)
    best_contestor = sorted(contestors, key=lambda indv: indv['fitness'], reverse=True)[0] #sorts contestors by descending order of fitness
  
    return best_contestor
    


In [130]:
def crossover(parent1, parent2, mutation_prob): # take two parents and create one child w/ a mutation probability
    
    rules_parent1 = [key for key in parent1.keys() if 'Rule' in key] #gets rules in parent genome
    
    child = {}

    for rule in rules_parent1:
        which_parent = random.randint(0,1)
        
        if which_parent == 0: # take parameters from parent1
            child[rule] = parent1[rule]
            
        else: #take parameters from parent 2
            child[rule] = parent2[rule]
    
    child['fitness'] = ()
    rules = [key for key in child.keys() if 'Rule' in key]

    #mutation
    if random.random() < mutation_prob:
        
        rule = random.choice(rules) #chooses a random rule to mutate
        r1 = parent1[rule]
        r2 = parent2[rule]

        if rule == 'Rule1':
            mean = int((r1+r2)/2) #mutates to average of values of rule 1
            child[rule] = mean

        else:
            mean_val_one = int((r1[0]+r2[0])/2)
            mean_val_two = int((r1[1]+r2[1])/2)
            child[rule] = [mean_val_one, mean_val_two]
    
    return child

In [132]:
def create_offspring(population, nim_size, mutation_prob): #gets new population of offspring
    offspring = []
    
    for i in range(OFFSPRING_SIZE):
        parent1 = tournament(population, nim_size) #gets parent 1
        parent2 = tournament(population, nim_size) #gets parent 2

        child = crossover(parent1, parent2, mutation_prob=mutation_prob) #crossover between parent 1 and 2 and creates one child
        
        offspring.append(child)
        
    return offspring

In [133]:
def get_next_generation(population): #sorts the population in descending order by fitness and returns the k ones with best fitness
    best_k_indv = sorted(population, key=lambda child: child['fitness'], reverse=True)[:POPULATION_SIZE]
    return best_k_indv


In [142]:
# GAME
nim_size = 5
GENERATIONS = 2

def evolution_agent(nim_size):
    population = init_population(nim_size) #creates population
    
    for _ in range(GENERATIONS):
    
        offspring = create_offspring(population, nim_size, 0.2) #creates a new offspring
    
        calculate_fitness(offspring, nim_size) #calculates fitness of offspring
    
        population = get_next_generation(offspring) #gets new population from best offspring
        
    best_indv = population[0]
    logging.debug(f"best player {best_indv}")
    
    return best_indv

evolution_agent(nim_size)

DEBUG:root:best player {'Rule1': 0, 'Rule2': [0, 4], 'Rule3': [0, 4], 'Rule4': [1, 4], 'Rule5': [0, 4], 'fitness': (0.0, 1.0, 1.0)}


{'Rule1': 0,
 'Rule2': [0, 4],
 'Rule3': [0, 4],
 'Rule4': [1, 4],
 'Rule5': [0, 4],
 'fitness': (0.0, 1.0, 1.0)}

### Task 3.3 - Minimax
https://realpython.com/python-minimax-nim/

In [19]:
def possible_new_states(nim_rows): #define all possible states 

  possible = []

  for row in range(len(nim_rows)) :

    for j in range(nim_rows[row]):
      new_state = deepcopy(nim_rows)

      new_state[row] = nim_rows[row] - j - 1
      possible.append(new_state)
  
  return possible



In [20]:
def evaluate(nim_rows, player): #evaluate if the game is finished
  if (sum(nim_rows) == 0):
    return -1 if player == 0 else 1 
  else: return None

In [21]:
def minimax(nim_rows, player):

  score = evaluate(nim_rows, player)

  if score != None:
    return score

  if (player == 0):

    scores = [minimax(new_state, player = 1) for new_state in possible_new_states(nim_rows)] 
    return max(scores)

  else:

    scores = [minimax(new_state, player = 0) for new_state in possible_new_states(nim_rows)] 
    return min(scores)



In [24]:
def best_move(nim_rows, player):

  if player == 0:
    new_player = 1

    return max(
      (minimax(new_state, new_player), new_state)
        for new_state in possible_new_states(nim_rows)
    )
  else:
    new_player = 0

  return min(
        (minimax(new_state, new_player), new_state)
        for new_state in possible_new_states(nim_rows)
    )

In [92]:
def minimax_pruning(state, is_maximizing, alpha=-1, beta=1):
    
    if (score := evaluate(state, is_maximizing)) is not None:
        return score

    scores = []
    
    for new_state in possible_new_states(state):
        scores.append(
            score := minimax_pruning(new_state, not is_maximizing, alpha, beta)
        )
        if is_maximizing:
            alpha = max(alpha, score)
        else:
            beta = min(beta, score)
            
        if beta <= alpha:
            break
            
    return (max if is_maximizing else min)(scores)
    

In [95]:
def best_move_pruning(state, player):
    if player == 0:
        return max(
            (minimax_pruning(new_state, is_maximizing=False), new_state)
            for new_state in possible_new_states(state)
        )
    else:
        return min(
            (minimax_pruning(new_state, is_maximizing=True), new_state)
            for new_state in possible_new_states(state)
        )

In [97]:
best_move_pruning([1,3,5], 1)

(-1, [1, 3, 2])

In [99]:
def nimply_move(current_state, new_state):
    
    diff = 0
    row = len(current_state) #invalid row
    
    for i in range(len(current__state)):
        
        if current_state[i] != new_state[i]:
            diff = current_state[i] - new_state[i]
            row = i
            
    return Nimply(row, diff)

### Task 3.4 - Reinforcement Learning
Q-Learning

In [ ]:
 class QLearner:
    #hashmap for the q function
    #learning rate and discount factor
    def __init__(self):
        self.q = 